In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using Turi Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Importing and reading data

In [4]:
import pandas as pd
from math import log, sqrt
from sklearn import linear_model  # using scikit-learn



* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

In [14]:
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':float, 'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}

sales = pd.read_csv('kc_house_data.csv', dtype=dtype_dict)
testing = pd.read_csv('wk3_kc_house_test_data.csv', dtype=dtype_dict)
training = pd.read_csv('wk3_kc_house_train_data.csv', dtype=dtype_dict)
validation = pd.read_csv('wk3_kc_house_valid_data.csv', dtype=dtype_dict)
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']
sales['floors_square'] = sales['floors']*sales['floors']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in Turi Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [5]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

model_all = linear_model.Lasso(alpha=5e2, normalize=True) # set parameters
model_all.fit(sales[all_features], sales['price']) # learn weights

Lasso(alpha=500.0, normalize=True)

Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

**QUIZ QUESTION According to this list of weights, which of the features have been chosen?**

In [10]:
lasso_features_dic = dict()
for i,feature in enumerate(all_features):
    lasso_features_dic[feature]=model_all.coef_[i]
lasso_features_dic

{'bedrooms': 0.0,
 'bedrooms_square': 0.0,
 'bathrooms': 0.0,
 'sqft_living': 134.43931395541435,
 'sqft_living_sqrt': 0.0,
 'sqft_lot': 0.0,
 'sqft_lot_sqrt': 0.0,
 'floors': 0.0,
 'floors_square': 0.0,
 'waterfront': 0.0,
 'view': 24750.00458560947,
 'condition': 0.0,
 'grade': 61749.10309070815,
 'sqft_above': 0.0,
 'sqft_basement': 0.0,
 'yr_built': -0.0,
 'yr_renovated': 0.0}

In [13]:
model_all.get_params(['deep'])

{'alpha': 500.0,
 'copy_X': True,
 'fit_intercept': True,
 'max_iter': 1000,
 'normalize': True,
 'positive': False,
 'precompute': False,
 'random_state': None,
 'selection': 'cyclic',
 'tol': 0.0001,
 'warm_start': False}

In [93]:
def sci(value):
    return('{:.5e}'.format(value))

# Selectin L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [15]:
testing['sqft_living_sqrt'] = testing['sqft_living'].apply(sqrt)
testing['sqft_lot_sqrt'] = testing['sqft_lot'].apply(sqrt)
testing['bedrooms_square'] = testing['bedrooms']*testing['bedrooms']
testing['floors_square'] = testing['floors']*testing['floors']

training['sqft_living_sqrt'] = training['sqft_living'].apply(sqrt)
training['sqft_lot_sqrt'] = training['sqft_lot'].apply(sqrt)
training['bedrooms_square'] = training['bedrooms']*training['bedrooms']
training['floors_square'] = training['floors']*training['floors']

validation['sqft_living_sqrt'] = validation['sqft_living'].apply(sqrt)
validation['sqft_lot_sqrt'] = validation['sqft_lot'].apply(sqrt)
validation['bedrooms_square'] = validation['bedrooms']*validation['bedrooms']
validation['floors_square'] = validation['floors']*validation['floors']

In [20]:
l1_penalty = np.logspace(1, 7, num=13)
RSS_val=[]
for l1_ in l1_penalty:
    model = linear_model.Lasso(alpha=l1_, normalize=True)
    X = training[all_features]
    y = training['price']
    X_val = validation[all_features]
    y_val = validation['price']
    model.fit(X,y) # learn weights on training set 
    y_hat_val = model.predict(X_val)
    RSS = sum((y_val-y_hat_val)**2)
    RSS_val.append(RSS)

**QUIZ QUESTION: What was the best value for the `l1_penalty`?**

In [21]:
best_l1 = l1_penalty[RSS_val.index(np.min(RSS_val))]
best_l1

10.0

In [22]:
best_l1 = l1_penalty[RSS_val.index(np.min(RSS_val))]
X = training [all_features]
X_test = testing [all_features]
y = training['price']
y_test = testing ['price']
# initialize model name & parameters
model = linear_model.Lasso(alpha=best_l1, normalize=True)
# fit model
model.fit(X, y)
y_hat_test = model.predict(X_test)
RSS_test = sum((y_test-y_hat_test)**2)
print("{:e}".format(RSS_test))


9.846740e+13


**QUIZ QUESTION: Also, using this value of L1 penalty, how many nonzero weights do you have?**

In [23]:
np.count_nonzero(model.coef_) + np.count_nonzero(model.intercept_)

15

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

You are going to implement a simple, two phase procedure to achieve this goal:

Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.

Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

10. Assign 7 to the variable `max_nonzeros`.

11. Exploring large range of l1_penalty

For l1_penalty in np.logspace(1, 4, num=20):

Fit a regression model with a given l1_penalty on TRAIN data. Add `alpha=l1_penalty` and `normalize=True` to the parameter list.

Extract the weights of the model and count the number of nonzeros. Take account of the intercept as we did in #8, adding 1 whenever the intercept is nonzero. Save the number of nonzeros to a list.

## Exploring the larger range of values to find a narrow range with the desired sparsity

12. Out of this large range, we want to find the two ends of our desired narrow range of l1_penalty. At one end, we will have l1_penalty values that have too few non-zeros, and at the other end, we will have an l1_penalty that has too many non-zeros.

More formally, find:

The largest l1_penalty that has more non-zeros than ‘max_nonzeros’ (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)Store this value in the variable ‘l1_penalty_min’ (we will use it later)

The smallest l1_penalty that has fewer non-zeros than ‘max_nonzeros’ (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)Store this value in the variable ‘l1_penalty_max’ (we will use it later)

Hint: there are many ways to do this, e.g.:

Programmatically within the loop above

Creating a list with the number of non-zeros for each value of l1_penalty and inspecting it to find the appropriate boundaries.

In [68]:
max_nonzeros = 7
# max_nonzero_id = np.sort((-model.coef_).argsort()[:max_nonzeros])
# max_nonzero_features = [all_features[i] for i in max_nonzero_id]
l1_penalty_list = np.logspace(1, 4, num=20) 
number_of_nonzeros= dict()
for l1_penalty in l1_penalty_list:
    model = linear_model.Lasso(alpha=l1_penalty, normalize=True)
    X = training[all_features]
    y = training['price']
    model.fit(X,y) # learn weights on training set 
    number_of_nonzeros [l1_penalty] = np.count_nonzero(model.coef_) + np.count_nonzero(model.intercept_)


In [ ]:
l1_penalty_min = 183.29807108324357


In [73]:
for key,value in number_of_nonzeros.items():
    if value > max_nonzeros:
        min_l1_penalty = key
        continue
    if value < max_nonzeros:
        max_l1_penalty = key
        break

**Quiz Question: What values did you find for l1_penalty_min and l1_penalty_max?**



In [75]:
min_l1_penalty,max_l1_penalty

(127.42749857031335, 263.6650898730358)

In [72]:
number_of_nonzeros

{10.0: 15,
 14.38449888287663: 15,
 20.6913808111479: 15,
 29.76351441631318: 15,
 42.81332398719393: 13,
 61.58482110660264: 12,
 88.58667904100822: 11,
 127.42749857031335: 10,
 183.29807108324357: 7,
 263.6650898730358: 6,
 379.26901907322497: 6,
 545.5594781168514: 6,
 784.7599703514607: 5,
 1128.8378916846884: 3,
 1623.776739188721: 3,
 2335.7214690901214: 2,
 3359.818286283781: 1,
 4832.930238571752: 1,
 6951.927961775606: 1,
 10000.0: 1}

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 
We will now explore the narrow region of `l1_penalty` values we found:

Exploring narrower range of l1_penalty

We now explore the region of l1_penalty we found: between ‘l1_penalty_min’ and ‘l1_penalty_max’. We look for the L1 penalty in this range that produces exactly the right number of nonzeros and also minimizes RSS on the VALIDATION set.

For l1_penalty in np.linspace(l1_penalty_min,l1_penalty_max,20):

Fit a regression model with a given l1_penalty on TRAIN data. As before, use "alpha=l1_penalty" and "normalize=True".

Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity equal to ‘max_nonzeros’. (Again, take account of the intercept when counting the number of nonzeros.)

In [97]:
l1_penalty_values = np.linspace(min_l1_penalty,max_l1_penalty,20)
RSS_val_narrow = dict()
for l1_penalty  in l1_penalty_values:
    model = linear_model.Lasso(alpha=l1_penalty, normalize=True)
    X = training[all_features]
    y = training['price']
    model.fit(X,y) # learn weights on training set 
    X_val = validation[all_features]
    y_val = validation['price']
    y_hat_val = model.predict(X_val)
    RSS = sum((y_val-y_hat_val)**2)
    RSS_val_narrow [l1_penalty] = [np.count_nonzero(model.coef_) + np.count_nonzero(model.intercept_),RSS]

 **Quiz Question: What value of l1_penalty in our narrow range has the lowest RSS on the VALIDATION set and has sparsity equal to ‘max_nonzeros’?**

In [95]:
RSS_val_narrow

{127.42749857031335: [10, '4.35375e+14'],
 134.5978981125619: [10, '4.37009e+14'],
 141.76829765481045: [8, '4.38236e+14'],
 148.938697197059: [8, '4.39159e+14'],
 156.10909673930755: [7, '4.40037e+14'],
 163.2794962815561: [7, '4.40777e+14'],
 170.44989582380464: [7, '4.41567e+14'],
 177.6202953660532: [7, '4.42406e+14'],
 184.79069490830176: [7, '4.43297e+14'],
 191.96109445055032: [7, '4.44240e+14'],
 199.13149399279888: [7, '4.45231e+14'],
 206.3018935350474: [6, '4.46269e+14'],
 213.47229307729594: [6, '4.47113e+14'],
 220.6426926195445: [6, '4.47998e+14'],
 227.81309216179307: [6, '4.48925e+14'],
 234.98349170404163: [6, '4.49892e+14'],
 242.1538912462902: [6, '4.50901e+14'],
 249.32429078853872: [6, '4.51952e+14'],
 256.49469033078725: [6, '4.53044e+14'],
 263.6650898730358: [6, '4.54177e+14']}

In [98]:
min_RSS = 1e100
for key,value in RSS_val_narrow.items():
    if value[0]==7 and value[1]<min_RSS:
        min_RSS = value[1]
        best_l1_penalty = key
print(best_l1_penalty)

156.10909673930755


**Quiz Question: What features in this model have non-zero coefficients?**

In [99]:
max_nonzeros = 7
model = linear_model.Lasso(alpha=best_l1_penalty, normalize=True)
X = training[all_features]
y = training['price']
model.fit(X,y) # learn weights on training set 
max_nonzero_id = np.sort((-model.coef_).argsort()[:max_nonzeros])
max_nonzero_features = [all_features[i] for i in max_nonzero_id]
max_nonzero_features

['bedrooms',
 'bathrooms',
 'sqft_living',
 'waterfront',
 'view',
 'grade',
 'sqft_basement']